In [1]:
import numpy as np # 数据处理最重要的模块
import pandas as pd # 数据处理最重要的模块
import scipy.stats as stats # 统计模块
import scipy
# import pymysql  # 导入数据库模块

from datetime import datetime # 时间模块
import statsmodels.formula.api as smf  # OLS regression

# import pyreadr # read RDS file

from matplotlib import style
import matplotlib.pyplot as plt  # 画图模块
import matplotlib.dates as mdates


from matplotlib.font_manager import FontProperties # 作图中文
from pylab import mpl
#mpl.rcParams['font.sans-serif'] = ['SimHei']
#plt.rcParams['font.family'] = 'Times New Roman'


#输出矢量图 渲染矢量图
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from IPython.core.interactiveshell import InteractiveShell # jupyter运行输出的模块
#显示每一个运行结果
InteractiveShell.ast_node_interactivity = 'all'

#设置行不限制数量
#pd.set_option('display.max_rows',None)

#设置列不限制数量
pd.set_option('display.max_columns', None)

# Stock Market Return (Excess Return)

In [3]:
from pandas.tseries.offsets import MonthEnd # 月末
Market_ret = pd.read_csv('Marketret_mon_stock2024.csv')
Market_ret['month'] = pd.to_datetime(Market_ret['month'], format='%b %Y') + MonthEnd(0)
Market_ret.set_index('month', inplace=True)
Market_ret.sort_index(inplace=True)
Market_ret = Market_ret.drop(columns=['Unnamed: 0'])
Market_ret

,MarketR,MarketR_e,rfmonth,ret,ret_e,marketret3,marketret6,marketret12,Q
month,,,,,,,,,
1991-01-31,0.029998,0.036554,0.006930,0.023068,0.029624,-0.084127,-0.305662,0.254049,1991 Q1
1991-02-28,0.010203,0.021860,0.006930,0.003273,0.014930,-0.183573,-0.384745,0.241492,1991 Q1
1991-03-31,-0.099663,-0.060425,0.006930,-0.106593,-0.067355,-0.252928,-0.445049,0.288857,1991 Q1
1991-04-30,-0.079652,-0.031449,0.006651,-0.086303,-0.038100,-0.234776,-0.394937,0.691749,1991 Q2
1991-05-31,-0.074521,0.005375,0.006092,-0.080613,-0.000717,-0.236294,0.181673,1.542701,1991 Q2
...,...,...,...,...,...,...,...,...,...
2024-08-31,-0.031468,-0.033685,0.001118,-0.032586,-0.034803,0.151050,NaN,NaN,2024 Q3
2024-09-30,0.195992,0.227078,0.001118,0.194874,0.225960,0.200147,NaN,NaN,2024 Q3
2024-10-31,-0.003409,0.070220,0.001118,-0.004527,0.069102,0.000228,NaN,NaN,2024 Q4


# Inflation

In [5]:
inflation = pd.read_csv('inflation.csv')
inflation['month'] = pd.to_datetime(inflation['month'],format='%Y/%m/%d')
inflation.set_index('month',inplace=True)
inflation

,cpi
month,
1987-01-31,5.1
1987-02-28,5.4
1987-03-31,5.8
1987-04-30,6.7
1987-05-31,7.6
...,...
2025-03-31,-0.1
2025-04-30,-0.1
2025-05-31,-0.1


# 月度数据的预测

A simple linear regression of an asset return on one or a few lagged predictors of interest is the most popular econometric approach for testing for return predictability. For simplicity, consider a univariate predictive regression of the period- $(t + 1)$ stock market return $r_{t+1}$ on a single predictor variable $x_t$:
$$
r_{t+1}=\alpha+\beta x_{t}+\varepsilon_{t+1}
$$
where $\varepsilon_{t+1}$ is a zero-mean, unpredictable disturbance term. When $x_t$ is the inflation rate, dividend yield, book-to-price ratio, or turnover. Many researchers find that $\beta$ is significantly different from zero; that is, there is in-sample evidence of stock market return predictability.

* H0:$\beta = 0$
* H1:$\beta \ne 0$(我们需要通过理论分析，得出$\beta$的符号)

## 模型： 通货膨胀与股票预期收益率

$$
r_{t+1} = \alpha + \beta*CPI_{t-1} + \varepsilon_{t+1}
$$

两个假设：

* H1： $\beta > 0$

* H2： $\beta < 0$

这是一个非常经典且重要的资产定价问题。通货膨胀与预期股票收益率之间的关系在金融学术界经过了数十年的广泛研究和激烈辩论，至今仍没有一个完全统一的定论。这恰恰是学术研究的魅力所在，即便是同一个问题，在不同的理论框架和实证检验下，也会得出看似矛盾的结论。

---

### **引言：经典的费雪假说**

在深入探讨正反两方观点之前，我们必须先了解最基础的理论出发点——**欧文·费雪（Irving Fisher）的费雪效应**。在其最简单的形式中，名义利率等于真实利率加上预期的通货膨胀率。

> 名义收益率 ≈ 真实收益率 + 预期通货膨胀率

如果我们将这个逻辑应用到股票上，并假设股票的预期真实收益率是相对稳定的（因为它由经济体的基本生产力、风险偏好等真实变量决定），那么预期名义股票收益率应该随着预期通货膨胀率的上升而上升。这意味着，股票作为对公司实物资产（厂房、设备、存货）所有权的代表，其名义价值应该能够对冲通胀带来的货币贬值，从而保持其真实价值。

这个直观且有力的理论，构成了两者正相关关系的核心基础。然而，大量的实证研究，尤其是在20世纪70年代之后，对这一经典假说提出了严峻的挑战。

---

### **一、 通货膨胀与预期股票收益率的正相关关系论述**

支持两者存在正相关关系的逻辑主要建立在股票的“实物资产”属性上。

1.  **理论逻辑：股票作为通胀的对冲工具（Inflation Hedge）**
    *   **基本原理**：股票代表的是对公司所有权的索取权，而公司的价值根本上来自于其拥有的实物资产（Real Assets）和未来的盈利能力。在通胀环境下，产品和服务的价格会上涨，公司的名义收入和名义利润也随之增加。同时，公司拥有的厂房、土地、存货等实物资产的名义价值也会重估上升。因此，从理论上讲，股票的内在价值应该与物价水平同步上涨，从而使其名义收益率能够补偿通货膨胀带来的损失，保证投资者的真实回报。
    *   **现金流折现模型视角**：在DCF模型中，通货膨胀会同时影响未来的名义现金流（分子）和名义折现率（分母）。如果通胀能够完全、无摩擦地传导至公司的产品售价和成本，并且名义折现率也根据费雪效应完全调整，那么股票的名义价格应该上升，以反映通胀。

2.  **实证支持：长期视角下的发现**
    *   早期的许多研究未能发现正相关关系，但这可能与研究的时间窗口有关。**[Boudoukh 和 Richardson (1993)](https://www.jstor.org/stable/2117566)** 的研究是一个重要的转折点。他们通过使用更长的时间序列数据（从19世纪至今），发现当考察的持有期越长（例如5年或更长），股票收益率与通货膨胀之间的正相关关系就越显著。
    *   **结论启示**：他们的研究表明，股票在短期内可能不是一个好的通胀对冲工具，但在长期来看，其作为实物资产的保值属性会逐渐显现。这暗示了市场在短期内可能存在定价偏差或反应不足，但在长期会回归其基本面价值。

---

### **二、 通货膨胀与预期股票收益率的负相关关系论述**

尽管正相关的理论直觉很强，但在20世纪下半叶，大量的实证研究（尤其是在美国市场）却惊人地发现两者之间存在稳健的**负相关**关系。这激发了金融学家们提出多种解释。

1.  **理论逻辑与解释**
    *   **税收效应（Tax Effect）**：这是最经典的解释之一。大多数国家的税法是基于**名义利润**和**名义资本利得**来征税的，而不是真实利润或真实资本利得。
        *   **折旧**：在通胀期间，基于历史成本计算的设备折旧额，其真实价值会缩水，导致账面上的应税利润被高估。
        *   **存货**：采用“先进先出”（FIFO）会计准则的公司，其销售成本会基于较早前购买的低成本存货，而收入则是按当前的高价格计算，这同样会虚增名义利润。
        *   **资本利得税**：投资者卖出股票时，需要为名义上的价差缴税。即使股票价格的上涨仅仅是为了补偿通胀（真实收益为零），投资者依然要为这部分“虚假”的收益纳税。
        这些税收上的扭曲效应，使得通货膨胀侵蚀了公司的真实税后利润和投资者的真实税后收益，从而降低了股票的吸引力，导致其价格下跌（即预期收益率与通胀负相关）。

    *   **货币幻觉（Money Illusion）**：由诺贝尔奖得主 **Modigliani 和 Cohn (1979)** 提出的著名假说。他们认为，投资者在评估股票价值时会犯两个系统性错误：
        1.  **错误的折现率**：投资者在用现金流折现模型估值时，倾向于使用名义利率去折现公司的真实现金流（未根据通胀调整的现金流），或者未能将公司因通胀而增加的债务价值（真实负债减少）考虑在内。这导致他们系统性地低估了股票的价值。
        2.  **对盈利质量的误判**：投资者看到了公司报告的名义利润在增长，但未能充分认识到，在高通胀环境下，维持同样生产能力所需的资本重置成本也在急剧上升。因此，报告的利润质量实际上是下降的。
        由于这种普遍存在的“货币幻觉”，当通胀上升时，市场会错误地压低股价。

    *   **代理假说（Proxy Hypothesis）**：这是另一位诺贝尔奖得主 **Fama (1981)** 提出的一个更精巧的理论。他认为，股票收益率和通货膨胀之间的负相关关系是“伪相关”（Spurious Correlation），两者本身没有直接的因果联系。真正的关系是：
        1.  股票收益率与**未来的实体经济活动**（如工业产出、GDP增长）**正相关**。因为股市是经济的晴雨表。
        2.  通货膨胀率与**未来的实体经济活动**则呈现**负相关**。这在当时被称为“滞胀”（Stagflation）现象，即高通胀往往预示着未来经济的衰退。
        3.  因此，股票收益率和通货膨胀率，因为它们都与同一个变量（未来经济活动）相关，但方向相反，从而表现出了表观上的负相关关系。通胀只是实体经济活动的“代理变量”。

    *   **风险溢价与不确定性（Risk Premium & Uncertainty）**：高通胀时期往往伴随着更高的经济不确定性和政策不确定性（例如，央行会如何应对）。为了补偿这种增加的系统性风险，投资者会要求更高的风险溢价。在短期内，要求更高的预期回报率意味着更低的当前价格。因此，通胀的上升通过增加风险溢价，导致了当前股价的下跌。

2.  **实证支持：里程碑式的研究**
    *   **[Fama 和 Schwert (1977)](https://doi.org/10.1016/0304-405X(77)90014-9)** 的研究是该领域的里程碑。他们检验了1953-1971年的美国数据，发现股票收益率与预期通胀、未预期通胀以及通胀变化都存在显著的负相关关系。他们明确指出，股票在那个时期是一个“坏”的通胀对冲工具，其表现甚至不如短期国债。这项研究的结论主导了学术界很长一段时间的看法。

---

### **总结与学术洞察**

总结一下：通货膨胀与预期股票收益率的关系远非“一对一”的简单映射。它取决于多种因素的复杂互动：

1.  **时间跨度**：在短期，税收效应、不确定性和市场非理性（如货币幻觉）可能占据主导，导致负相关。但在长期，股票作为实物资产的对冲属性可能最终显现，呈现正相关。
2.  **通胀的来源和性质**：由需求拉动的温和通胀（Demand-Pull Inflation）可能伴随着经济繁荣，对股价有利。而由成本推动的通胀（Cost-Push Inflation），如石油危机，则会损害企业利润，对股价不利。
3.  **制度因素**：税法的设计、会计准则的选择以及央行的政策反应框架，都会深刻影响这一关系。例如，如果税法改革为通胀进行指数化调整，那么负相关的税收效应就会减弱。
4.  **研究样本与时期**：20世纪70-80年代的“大滞胀”时期，负相关关系尤为明显。但在其他历史时期或不同国家，这种关系可能不那么稳定。

In [7]:
reg_data = pd.merge(Market_ret, inflation, left_index=True, right_index=True,how='left')
# 使用 pandas 的 merge 函数将两个DataFrame根据索引进行合并,且执行的是左连接操作
reg_data

,MarketR,MarketR_e,rfmonth,ret,ret_e,marketret3,marketret6,marketret12,Q,cpi
month,,,,,,,,,,
1991-01-31,0.029998,0.036554,0.006930,0.023068,0.029624,-0.084127,-0.305662,0.254049,1991 Q1,2.2
1991-02-28,0.010203,0.021860,0.006930,0.003273,0.014930,-0.183573,-0.384745,0.241492,1991 Q1,1.0
1991-03-31,-0.099663,-0.060425,0.006930,-0.106593,-0.067355,-0.252928,-0.445049,0.288857,1991 Q1,1.6
1991-04-30,-0.079652,-0.031449,0.006651,-0.086303,-0.038100,-0.234776,-0.394937,0.691749,1991 Q2,1.3
1991-05-31,-0.074521,0.005375,0.006092,-0.080613,-0.000717,-0.236294,0.181673,1.542701,1991 Q2,3.6
...,...,...,...,...,...,...,...,...,...,...
2024-08-31,-0.031468,-0.033685,0.001118,-0.032586,-0.034803,0.151050,NaN,NaN,2024 Q3,0.6
2024-09-30,0.195992,0.227078,0.001118,0.194874,0.225960,0.200147,NaN,NaN,2024 Q3,0.4
2024-10-31,-0.003409,0.070220,0.001118,-0.004527,0.069102,0.000228,NaN,NaN,2024 Q4,0.3


In [9]:
# 导出数据
reg_data.to_csv('reg_data.csv')
# save as excel
reg_data.to_excel('reg_data.xlsx')